## 281. Zigzag Iterator
- Description:
  <blockquote>
    Given two vectors of integers v1 and v2, implement an iterator to return their elements alternately.

  Implement the ZigzagIterator class:

      ZigzagIterator(List<int> v1, List<int> v2) initializes the object with the two vectors v1 and v2.
      boolean hasNext() returns true if the iterator still has elements, and false otherwise.
      int next() returns the current element of the iterator and moves the iterator to the next element.



  Example 1:

  Input: v1 = [1,2], v2 = [3,4,5,6]
  Output: [1,3,2,4,5,6]
  Explanation: By calling next repeatedly until hasNext returns false, the order of elements returned by next should be: [1,3,2,4,5,6].

  Example 2:

  Input: v1 = [1], v2 = []
  Output: [1]

  Example 3:

  Input: v1 = [], v2 = [1]
  Output: [1]



  Constraints:

      0 <= v1.length, v2.length <= 1000
      1 <= v1.length + v2.length <= 2000
      -231 <= v1[i], v2[i] <= 231 - 1



  Follow up: What if you are given k vectors? How well can your code be extended to such cases?

  Clarification for the follow-up question:

  The "Zigzag" order is not clearly defined and is ambiguous for k > 2 cases. If "Zigzag" does not look right to you, replace "Zigzag" with "Cyclic".

  Follow-up Example:

  Input: v1 = [1,2,3], v2 = [4,5,6,7], v3 = [8,9]
  Output: [1,4,8,2,5,9,3,6,7]


  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/zigzag-iterator/description/?envType=company&envId=coinbase&favoriteSlug=coinbase-all)

- Topics: Array, Queue,  Two-Pointers, 2D Array

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1
Two-Pointers 2D Array traversal Solution

We are asked to iterate the elements, while alternating the vectors.
One can imagine this as iterating over a two-dimension matrix, where each row represents an input vector.

    The idea is that we can employ two pointers for iteration: one pointed to the vector (denoted as p_vec), and the other pointed to the element within the vector (denoted as p_elem).

two pointers

As we can see from the above graph, the vector pointer (p_vec) will move in the zigzag way (more precisely cyclic way), i.e. once it reaches the last vector, it will start all over from the first vector.

The element pointer (p_elem) increments, only when the vector pointer finishes a cycle.

We give the priority to the vector pointer, i.e. we move the vector pointer first, then the element pointer.


Let K be the number of input vectors, although it is always two in the setting of the problem.
This variable becomes relevant, once the input becomes K vectors.

- Time Complexity: O(K)
  - For the next() function, at most it will take us K iterations to find a valid element output. Hence, its time complexity is O(K).
  - For the hasNext() function, its time complexity is O(1).

- Space Complexity: O(K)
  - For the next() function, we keep the references to all the input vectors in the variable self.vectors.
    As a result, we would need O(K) space for K vectors.
    In addition, we used some constant-space variables such as the pointers to the vector and the element.
    Hence, the overall space complexity for this function is O(K).
  - Note: we did not copy the input vectors, but simply keep references to them.


In [ ]:
from typing import List

class ZigzagIterator:
    def __init__(self, v1: List[int], v2: List[int]):
        self.vectors = [v1, v2]
        self.p_elem = 0   # pointer to the index of element
        self.p_vec = 0    # pointer to the vector
        # variables for hasNext() function
        self.total_num = len(v1) + len(v2)
        self.output_count = 0

    def next(self) -> int:
        iter_num = 0
        ret = None

        # Iterate over the vectors
        while iter_num < len(self.vectors):
            curr_vec = self.vectors[self.p_vec]
            
            if self.p_elem < len(curr_vec):
                ret = curr_vec[self.p_elem]

            iter_num += 1
            
            # circular iteration through the vectors array.
            # (self.p_vec + 1) increments this pointer to move to the next vector
            # % len(self.vectors) applies the modulo operation, which "wraps around" the pointer when it reaches the end of the vectors list
            self.p_vec = (self.p_vec + 1) % len(self.vectors)
            
            # increment the element pointer once iterating all vectors
            if self.p_vec == 0:
                self.p_elem += 1

            if ret is not None:
                self.output_count += 1
                return ret

        # no more element to output
        raise Exception


    def hasNext(self) -> bool:
        return self.output_count < self.total_num

# Your ZigzagIterator object will be instantiated and called as such:
# i, v = ZigzagIterator(v1, v2), []
# while i.hasNext(): v.append(i.next())

### Solution 2
Queue of Pointers

The above approach is not the most efficient when the input vectors are not of equal size.
For instance, for the input vectors of [1], [2, 3, 4, 5], we would waste some computing cycles to alternate the vector pointer, once we consume all the elements from the shorter vector.
The problem exacerbates when the number of input vectors grows.

One idea to alleviate the above problem is to keep a queue of pointers to the input vectors as shown in the following graph.

The queue functions in the following ways:

Initially, each input vector will have a corresponding pointer in the queue.

At each invocation of next() function, we pop out a pointer from the queue. With the pointer to the chosen vector, we further output an element from the vector.

If the vector still has some elements left, we append another pointer pointed to the vector at the end of the queue.
In this way, we alternate the order of vectors.

If all the elements in the chosen vector are already outputted, we will NOT append another pointer. As a result, the vector would be out of the scope of the iteration. Later we won't waste any effort to iterate over the vectors that are exhausted.

As to the hasNext() function, as long as there are still some pointers left in the queue, we would still have more elements to output.



Advantages of this method:
- First of all, we would achieve a constant time complexity for the next() function.
- The logics of implementation is much simplified and thus easy to read.



Time Complexity: O(1)

Space Complexity: O(K)
  - We use a queue to keep track of the pointers to the input vectors in the variable self.vectors.
    As a result, we would need O(K) space for K vectors.
  - Although the size of queue will reduce over time once we exhaust some shorter vectors, the space complexity for both functions is still O(K).


In [ ]:
from collections import deque
from typing import List

class ZigzagIterator:
    def __init__(self, v1: List[int], v2: List[int]):
        self.vectors = [v1, v2]
        self.queue = deque()
        
        for index, vector in enumerate(self.vectors):
            # <index_of_vector, index_of_element_to_output>
            if len(vector) > 0:
                self.queue.append((index, 0))

    def next(self) -> int:

        if self.queue:
            vec_index, elem_index = self.queue.popleft()
            
            next_elem_index = elem_index + 1
            
            if next_elem_index < len(self.vectors[vec_index]):
                # append the pointer for the next round
                # if there are some elements left
                self.queue.append((vec_index, next_elem_index))

            return self.vectors[vec_index][elem_index]

        # no more element to output
        raise Exception

    def hasNext(self) -> bool:
        return len(self.queue) > 0